In [1]:
import sys
sys.path.append('../bin/')
from load_data import *
import matplotlib.pyplot as plt
from classification import *
import time

In [2]:
input_data = 'mnist'
num_classes= 10
DATA_SHAPE = 28*28

In [3]:
if input_data == 'mnist':
    load_data = load_mnist
if input_data == 'cifar10':
    load_data = load_cifar10

In [4]:
x_train, targets_train, x_test, targets_test = load_data()

In [5]:
def make_plots(x, input_type):

    num = np.int(np.floor(np.sqrt(len(x))))
    if num > 8:
        num = 8
    if input_type=='mnist':
        fig, axes = plt.subplots(num,num,figsize=(num,num))
        plt.set_cmap('gray')
        k =0
        for j in range(num):
            for i in range(num):
                k+=1
                axes[i][j].imshow(x[k].reshape(28,28))
                axes[i][j].axis('off')
        
        plt.show()
        
    elif input_type=='cifar10':
        fig, axes = plt.subplots(num,num,figsize=(num,num))
        k =0
        for j in range(num):
            for i in range(num):
                k+=1
                img = np.swapaxes(np.reshape((x[k]),(32,32,3),'F'),0,1)
                axes[i][j].imshow(img, interpolation='bilinear')
                axes[i][j].axis('off')
        plt.show()
    return True


In [10]:
from sklearn.covariance import LedoitWolf, EmpiricalCovariance, OAS
import scipy.linalg as lg
import numpy.linalg as nlg
from sklearn.decomposition import PCA

def get_covariance(R,var,num):
    '''
    get covarinace estimation for specific number of components
    num: number of components
    R: matrix of eigenvectors
    var: array of eigenvalues
    '''
    fl   = len(var)
    var_ = var[0:num]
    R    = R[0:num]
    
    if num<fl:
        sigma2 = np.mean(var[num::])
    else: 
        sigma2 = 0.

    C_            = np.dot(R.T,np.dot(np.diag(var_), R))+np.eye(len(R.T))*sigma2
    Cinv          = lg.inv(C_)
    sign ,logdetC = nlg.slogdet(C_)
    
    return Cinv, logdetC


class CovarianceEstimator():

    def __init__(self,mode, label, masking):
        
        assert(mode in ['ML','OAS', 'LW','NERCOME'])
        self.mode   = mode
        self.label  = label
        self.masking= masking
    
    def diag_decomp(self):
        var,R     = lg.eigh(self.cov)
        indices   = np.argsort(var)[::-1]
        self.vars = var[indices]
        self.R    = R.T[indices]
        return self.vars, self.R
    
    def pca(self,data):
        
        num       = data.shape[1]
        pca       = PCA(svd_solver='full',n_components=num)
        pca.fit(data)
        self.pca_vars= pca.explained_variance_
        self.pca_R   = pca.components_
        

    def fit(self,data, mask_in, mask_out):
        
        if len(mask_in)==data.shape[1]:
            self.masking = False
            print('no zero variance pixels in data')
        
        if self.masking:
            data = data[:, mask_in]
        else:
            data[:,mask_out] = 0.01
        
        if self.mode =='ML':
            self.cov = EmpiricalCovariance().fit(data).covariance_
        elif self.mode =='OAS':
            self.cov = OAS().fit(data).covariance_
        elif self.mode =='LW':
            self.cov = LW().fit(data).covariance_
        elif mode == 'NERCOME':
            pass
        else: 
            raise ValueError
        
        if not self.masking:
            self.cov[mask_out,:] =0.
            self.cov[:,mask_out] =0.
            self.cov[mask_out,mask_out] = 1e-4
            
        self.mask_in = mask_in
        self.mask_out= mask_out
        self.mean = np.mean(data, axis=0)

        return True
    
    def compute_logdet(self):
        sign ,self.logdetC = nlg.slogdet(self.cov)
        return True
    
    def compute_inverse(self):
        self.Cinv = lg.inv(self.cov)
        return True
    
    def save(self, path):
        if not os.path.exists(path):
            os.makedirs(path)
        if self.masking:
            self.filename = os.path.join(path,'cov_estimate_%s_%d_masked.pkl'%(self.mode,self.label))
        else:   
            self.filename = os.path.join(path,'cov_estimate_%s_%d.pkl'%(self.mode,self.label))
        
        pkl.dump(self, open(self.filename,'wb'))
        
        return self.filename

In [11]:
def prepare_data(data, labels, num_classes):
    '''
    orderes data by labels and identifies zero variance pixels
    '''
    ordered_data = []
    masks_in     = []
    masks_out    = []
    for ii in range(num_classes):
        ind     = np.where(labels==ii)[0]
        d       = data[ind]
        mask_in = np.where(np.var(d, axis=0)>0.)[0]
        ordered_data+=[d]
        mask_out= np.where(np.var(d, axis=0)==0.)[0]
        masks_in+=[mask_in]
        masks_out+=[mask_out]
    
    return ordered_data, masks_in, masks_out

In [12]:
d_v, mv_in, mv_out = prepare_data(data=x_train,labels=targets_train, num_classes=num_classes)
d_t, mt_in, mt_out = prepare_data(data=x_test,labels=targets_test, num_classes=num_classes)

In [13]:
covs = []
for ii, d in enumerate(d_v):
    cov = CovarianceEstimator(mode='ML',label=ii,masking=False)
    cov.fit(d,mv_in[ii],mv_out[ii])
    cov.diag_decomp()
#     cov.compute_logdet()
#     cov.compute_inverse()
    filename = cov.save('../outputs/%s/covariance_estimator/'%input_data)
    print(filename)
    covs+=[cov]

../outputs/mnist/covariance_estimator/cov_estimate_ML_0.pkl
../outputs/mnist/covariance_estimator/cov_estimate_ML_1.pkl
../outputs/mnist/covariance_estimator/cov_estimate_ML_2.pkl
../outputs/mnist/covariance_estimator/cov_estimate_ML_3.pkl
../outputs/mnist/covariance_estimator/cov_estimate_ML_4.pkl
../outputs/mnist/covariance_estimator/cov_estimate_ML_5.pkl
../outputs/mnist/covariance_estimator/cov_estimate_ML_6.pkl
../outputs/mnist/covariance_estimator/cov_estimate_ML_7.pkl
../outputs/mnist/covariance_estimator/cov_estimate_ML_8.pkl
../outputs/mnist/covariance_estimator/cov_estimate_ML_9.pkl


In [14]:
def get_log_prob(logdet,Cinv,data,mean):
    d    = len(data)
    data = data-mean
    Cinv_d = np.einsum('jk,...k->...j',Cinv,data, optimize=True)
    logprob = -0.5*logdet-0.5*d*np.log((2*np.pi))-0.5*np.einsum('ij,ij->i',data, Cinv_d, optimize=True)
    return logprob

In [16]:
def classify(data,labels,covs, num_classes, num):
    acc = []
    for jj in range(num_classes):
        print(jj)
        indices = np.where(labels==jj)[0]
        dd      = data[indices]
        logprob=[]
        for ii,cov in enumerate(covs):
            if num == dd.shape[1]:
                Cinv    = cov.Cinv
                logdetC = cov.logdetC
            else:
                Cinv, logdetC = get_covariance(cov.R,cov.vars,num)
            if cov.masking:
                d_      = dd[:,cov.mask_in]
            else:
                d_      = np.ones_like(dd)*0.01
                d_[:,cov.mask_in] = dd[:,cov.mask_in]
            logprob_= get_log_prob(logdetC,Cinv,d_,cov.mean)
            logprob+=[logprob_]
        acc+=[len(np.where(np.argsort(np.asarray(logprob),axis=0)[-1]==jj)[0])/len(dd)]
    return np.asarray(acc)

In [17]:

accuracy = classify(x_test,targets_test,covs, num_classes, num=200)

0
1
2
3
4
5
6
7
8
9


(array([ 0.97142857,  0.91982379,  0.93895349,  0.9039604 ,  0.92464358,
         0.85650224,  0.93110647,  0.8540856 ,  0.9137577 ,  0.89494549]),
 array([ 0.96122449,  0.91894273,  0.97868217,  0.92178218,  0.95723014,
         0.90919283,  0.91858038,  0.92898833,  0.82340862,  0.76808722]))

In [18]:
print(accuracy)

[ 0.97142857  0.91982379  0.93895349  0.9039604   0.92464358  0.85650224
  0.93110647  0.8540856   0.9137577   0.89494549]
